In [1]:
using JuMP, GLPK
using Printf

In [3]:
c1 = [1, 4]
c2 = [2, 3]
b2 = [-2; -3]
A1= [1 -3; -1 -3]
A2= [1 -2; -1 -1]
M = -1000;

In [4]:
main = Model(GLPK.Optimizer)


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [5]:
@variable(main, 0 <= x[1:2], Int)
@variable(main, -1000 <= θ) # \theta is a continouos variable
@objective(main, Min, c1' * x + θ);

In [6]:
function sub(x)
    sub = Model(GLPK.Optimizer)
    @variable(sub, 0 <= y[1:2])
    @objective(sub, Min, c2' * y)
    @constraint(sub, A1 * x + A2 * y .<= b2)
    optimize!(sub)
    o = objective_value(sub)
    y = value.(y)
    all_con = all_constraints(sub, AffExpr, MOI.LessThan{Float64})
    λ = dual.(all_con)
    return Dict("o"=>o, "y" => y, "λ" => λ)
end

sub (generic function with 1 method)

In [7]:
sub([0, 0])

Dict{String, Any} with 3 entries:
  "λ" => [-0.333333, -2.33333]
  "o" => 7.66667
  "y" => [1.33333, 1.66667]

In [11]:
function print_iteration(k, args...)
    f(x) = Printf.@sprintf("%12.4e", x)
    println(lpad(k, 9), " ", join(f.(args), " "))
    return
end

print_iteration (generic function with 1 method)

In [12]:
println("k  lowerbound  upperbound      gap")
for k in 1:10
    optimize!(main)
    lb = objective_value(main)
    x_k = value.(x)
    ub = c1' * x_k + c2' * sub(x_k)["y"]
    gap = (ub - lb)/ ub
    print_iteration(k, lb, ub, gap)
    if gap < 0.0000001
        println("**We solved the problem to optimimality.**")
        break
    end
    benderscut = @constraint(main, θ >= sub(x_k)["o"] - (sub(x_k)["λ"])' * A1 * (x .- x_k))
    @info "We are adding this bender cut $(benderscut)"
end

k  lowerbound  upperbound      gap


        1  -1.0000e+03   7.6667e+00   1.3143e+02


        2  -4.9600e+02   1.2630e+03   1.3927e+00
        3  -1.0800e+02   8.8800e+02   1.1216e+00
        4   4.0000e+00   4.0000e+00   0.0000e+00
**We solved the problem to optimimality.**


┌ Info: We are adding this bender cut 2 x[1] + 8 x[2] + θ ≥ 7.666666666666666
└ @ Main /home/vahid/code/JuMP-workspace/benders_decomposition.ipynb:14
┌ Info: We are adding this bender cut -1.5 x[1] + 4.5 x[2] + θ ≥ 3
└ @ Main /home/vahid/code/JuMP-workspace/benders_decomposition.ipynb:14
┌ Info: We are adding this bender cut θ ≥ 0
└ @ Main /home/vahid/code/JuMP-workspace/benders_decomposition.ipynb:14
